# 자동화된 Machine Learning 사용

여러 종류의 기계 학습 알고리즘을 사용하여 모델을 학습시킬 수 있습니다. 그리고 특정 데이터 및 예측 요구 사항을 기준으로 가장 효율적인 알고리즘을 결정하기가 쉽지 않은 경우도 있습니다. 또한 정규화, 누락된 기능 대치 등의 기술을 사용해 학습 데이터를 전처리하면 모델의 예측 성능을 크게 개선할 수 있습니다. 요구 사항에 *가장 적합한* 모델을 찾으려면 여러 알고리즘 및 전처리 변환 조합을 적용해 보아야 할 수 있는데, 그러려면 시간이 많이 걸리며 컴퓨팅 리소스도 많이 필요합니다.

Azure Machine Learning을 사용하면 다양한 알고리즘 및 전처리 옵션을 사용하여 학습된 모델을 비교하는 과정을 자동화할 수 있습니다. SDK 또는 [Azure Machine Learning Studio](https://ml/azure.com)의 시각적 인터페이스에서 이 기능을 활용할 수 있습니다. SDK를 사용하면 자동화된 Machine Learning 실험의 설정을 더 자세하게 제어할 수 있지만, 시각적 인터페이스가 더 사용하기 쉽습니다.

## 시작하기 전 주의 사항

이 Notebook의 코드를 실행하려면 최신 버전의 **azureml-sdk** 및 **azureml-widgets** 패키지 외에 **azureml-train-automl** 패키지도 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

In [ ]:
!pip show azureml-train-automl

## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 데이터 준비

자동화된 Machine Learning용 학습 스크립트는 만들지 않아도 되지만, 학습 데이터는 로드해야 합니다. 여기서는 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용하며, 해당 데이터 세트를 두 개(학습용과 모델 유효성 검사용)로 분할합니다.

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


# 데이터 세트를 학습 하위 세트와 유효성 검사 하위 세트로 분할
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

## 컴퓨팅 리소스 준비

클라우드 컴퓨팅의 이점 중 하나는 요청 시에 크기를 조정할 수 있다는 것입니다. 따라서 자동화된 Machine Learning 실험의 여러 하위 실행을 병렬로 처리하기에 충분한 컴퓨팅 리소스를 프로비전할 수 있습니다.

다음 코드를 사용하여 Azure Machine Learning 컴퓨팅 클러스터를 지정합니다. 이 클러스터는 아직 없으면 자동으로 생성됩니다.

> **중요**: 컴퓨팅 클러스터를 실행하기 전에 아래 코드에서 *your-compute-cluster*를 컴퓨팅 클러스터의 이름으로 변경하세요! 클러스터 이름은 2~16자 사이의 전역으로 고유한 이름이어야 합니다. 유효한 문자는 영문자, 숫자 및 문자입니다.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **참고**: 컴퓨팅 인스턴스와 클러스터는 표준 Azure 가상 머신 이미지를 기반으로 합니다. 이 연습에서는 비용과 성능 간 최적의 균형을 달성하기 위해 *Standard_DS11_v2* 이미지를 사용하는 것이 좋습니다. 구독의 할당량이 적어 이 이미지를 포함할 수 없는 경우 대체 이미지를 선택할 수 있습니다. 그러나 큰 이미지는 높은 비용을 야기할 수 있고 작은 이미지는 작업을 완료하는 데 충분하지 않을 수 있으므로 신중히 선택하는 것이 좋습니다. Azure 관리자에게 요청하여 할당량을 늘릴 수도 있습니다.

## 자동화된 Machine Learning 구성

이제 자동화된 Machine Learning 실험을 구성할 수 있습니다.

가장 중요한 구성 설정 중 하나는 모델 성능 평가 기준으로 사용되는 메트릭입니다. 다음과 같은 코드를 사용하면 특정 모델 작업 유형(분류 또는 회귀)에 대해 자동화된 Machine Learning에서 계산하는 메트릭 목록을 검색할 수 있습니다.

In [ ]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

최적화할 메트릭(이 예제에서는 *AUC_weighted*)을 결정한 후에는 자동화된 Machine Learning 실행을 구성할 수 있습니다. 이렇게 하려면 대상 메트릭과 기타 다양한 옵션(예: 사용할 데이터, 시도할 조합 수 등)을 지정하는 AutoML 구성이 필요합니다.

> **참고**: 이 예제에서는 실험을 4회 반복으로 제한하여 소요 시간을 줄입니다. 실제로 더 많은 반복을 시도할 수 있습니다.

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=4,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

## 자동화된 Machine Learning 실험 실행

이제 실험을 실행할 준비가 되었으므로 자동화된 Machine Learning 실험을 실행해 보겠습니다.

> **참고**: 파이프라인을 실행하려면 시간이 다소 걸릴 수 있습니다.

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'mslearn-diabetes-automl-sdk')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## 하위 실행 세부 정보 보기

실험이 완료되면 위젯에서 출력을 확인한 다음 최적 결과를 생성한 실행을 클릭하여 해당 세부 정보를 살펴봅니다.

그런 다음 링크를 클릭하여 Azure Machine Learning Studio에서 실험 세부 정보와 전체 실험 세부 정보를 확인하고, 마지막으로 최적 결과를 생성한 개별 실험의 세부 정보를 확인합니다. 이 세부 정보에는 생성된 모델의 성능에 대한 여러 정보가 포함되어 있습니다.

SDK를 사용하여 모든 하위 실행을 검색한 후 실험의 메트릭을 확인할 수도 있습니다.

In [ ]:
for run in automl_run.get_children():
    print('Run ID', run.id)
    for metric in run.get_metrics():
        print('\t', run.get_metrics(metric))

## 최적 실행 확인

최적 실행을 검색하고 다음과 같은 실행 세부 정보를 확인할 수 있습니다.

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

성능이 가장 우수한 모델을 확인한 후에는 마지막으로 해당 모델을 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

# 모델 등록
best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# 등록된 모델 목록 표시
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **추가 정보**: 자동화된 Machine Learning에 대한 자세한 내용은 [Azure ML 설명서](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train)를 참조하세요.